In [80]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import xlsxwriter
import datetime

In [81]:
# ususaarios de base de datos local

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"
# puerto = '3306'
# esquema = 'EspoCRM2'
# plugin_autenticacion = 'mysql_native_password'

# usuarios de base de datos Simtastic ##########################################

# usuario = 'siempreread'
##contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "EspoCRM2"
plugin_autenticacion = "mysql_native_password"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

In [86]:
sql = "select cas.number as 'Id',trim(con.rut) as 'Rut Paciente',trim(con.rut_asegurado) as 'Rut Titular',con.relacion_con_asegurado as 'Relación con asegurado', cas.area as 'Prestacion',"
sql2 = " cas.created_at as 'Fecha Gestión', con.empleador AS 'Empleador', cas.channel as 'Canal', trim(con.campana_paciente) as 'Campaña', con.holding as 'holding'"
sql3 = "from EspoCRM2.case cas left join EspoCRM2.contact con on cas.contact_id = con.id where con.campana_paciente like '%vida security%' AND cas.deleted = 0 and cas.status <> 'Cancelled' order by cas.number asc;"

# archivo de salida modificar segun archivo
archivo = "C:/Users/jmartinez/Desktop/Informes/Metlife/bhp.xlsx"

fin_sql = sql + sql2 + sql3

In [87]:
conchaytoro = pd.read_sql_query(fin_sql, motor_mysql_mariadb)



conchaytoro.head(10)


,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña,holding
0,77093,15557293-0,15557293-0,AS,Siempre - Medicina,2021-03-22 12:49:49,SALFA GESTION S.A.,Llamada telefónica,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.
1,78768,17809129-8,17809129-8,AS,Siempre - Deportiva,2021-03-27 13:18:47,ARQUITECTURA E INGENIERIA ACONCAGUA SPA,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.
2,78786,15557293-0,15557293-0,AS,Siempre - Deportiva,2021-03-27 13:42:59,SALFA GESTION S.A.,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.
3,78801,17809129-8,17809129-8,AS,Siempre - Deportiva,2021-03-27 14:08:31,ARQUITECTURA E INGENIERIA ACONCAGUA SPA,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.
4,82413,8456211-4,8456211-4,AS,Siempre - Medicina,2021-04-09 13:34:07,EMPRESA DE MONTAJES INDUSTRIALES SALFA S A,Llamada telefónica,Vida Security,SALFA GESTION INGENIERIA S.A.
5,82610,16411587-9,16411587-9,AS,Siempre - Medicina,2021-04-09 18:51:48,SALFA GESTION S.A.,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.
6,87323,15639642-7,15639642-7,AS,Siempre - Medicina,2021-04-20 20:43:09,ACONCAGUA GESTION INMOBILIARIA S.A.,Chat Online,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.
7,87869,15639642-7,15639642-7,AS,Siempre - Medicina,2021-04-21 21:33:11,ACONCAGUA GESTION INMOBILIARIA S.A.,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.
8,87896,11915018-3,11915018-3,AS,Siempre - Medicina,2021-04-21 21:55:14,SALFA GESTION INGENIERIA S.A.,Llamada telefónica,Vida Security,SALFA GESTION INGENIERIA S.A.
9,87917,15639642-7,15639642-7,AS,Siempre - Medicina,2021-04-21 22:54:36,ACONCAGUA GESTION INMOBILIARIA S.A.,Llamada telefónica,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.


In [88]:
conchaytoro.shape

(3817, 10)

In [89]:
relacion = {
    "HI": "Hijo/a",
    "AS": "Titular",
    "CO": "Cónyuge",
    "P": "Padre",
    "O": "Otros",
    "M": "Madre",
}
conchaytoro.replace({"Relación con asegurado": relacion}, inplace=True)

prestaciones = {
    "Deporte": "Programa Deportivo",
    "Nutrición": "Programa Nutricional",
    "Pcte. Crónico": "Programa Enfermería",
    "Orientación": "Orientación Hospitalaria",
    "Psicología": "Programa de Psicología",
    "Psicología": "Programa de Psicología",
    "Medicina": "Doctor Online",
    "Deporte": "Programa Deportivo",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Medicina": "Doctor Online",
    "Psicología": "Programa de Psicología",
    "Nutrición": "Programa Nutricional",
    "Deporte": "Programa Deportivo",
    "Fidelización": "Fidelización",
    "Orientación": "Orientación Hospitalaria",
    "Pcte. Crónico": "Programa Enfermería",
    "Medicina": "Doctor Online",
    "Agenda": "Orientación Hospitalaria",
    "Fidelización y SAC": "Orientación Hospitalaria",
    "Ingreso dep.  seguro": "Programa Deportivo",
    "Clínica del Sueño": "Programa Enfermería",
    "Ingreso dep. seguro": "Programa Deportivo",
    "APSSex": "Programa de Psicología",
    "Kinesiología HC": "Programa Deportivo",
    "Siempre - Medicina": "Doctor Online",
    "Siempre - Orientación": "Orientación Hospitalaria",
    "Siempre - Agenda": "Orientación Hospitalaria",
    "Siempre - Deportiva": "Programa Deportivo",
    "Siempre - Psicología": "Programa de Psicología",
    "Siempre - Nutrición": "Programa Nutricional",
    "Siempre  - Front": "Orientación Hospitalaria",
    "Siempre - Clínica del Sueño": "Programa Enfermería",
    "Siempre - Enfermería": "Programa Enfermería",
    "Telemedicina": "Doctor Online",
    "Nutricion": "Programa Nutricional",
    "Front": "Orientación Hospitalaria",
    "Deportivo": "Programa Deportivo",
}
conchaytoro.replace({"Prestacion": prestaciones}, inplace=True)


conchaytoro["derty"] = pd.to_datetime(
    conchaytoro["Fecha Gestión"], format="dd-md-yyyy h:mm"
)
conchaytoro.derty = conchaytoro.derty.dt.strftime("%d-%m-%Y %H:%M")
# conchaytoro['Fecha Gestión'] = conchaytoro['Fecha Gestión'].apply(lambda _: datetime.strptime(_,"%d-%m-%Y, %H:%M"))

In [90]:
conchaytoro.head(50)

,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña,holding,derty
0,77093,15557293-0,15557293-0,Titular,Doctor Online,2021-03-22 12:49:49,SALFA GESTION S.A.,Llamada telefónica,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.,22-03-2021 12:49
1,78768,17809129-8,17809129-8,Titular,Programa Deportivo,2021-03-27 13:18:47,ARQUITECTURA E INGENIERIA ACONCAGUA SPA,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.,27-03-2021 13:18
2,78786,15557293-0,15557293-0,Titular,Programa Deportivo,2021-03-27 13:42:59,SALFA GESTION S.A.,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.,27-03-2021 13:42
3,78801,17809129-8,17809129-8,Titular,Programa Deportivo,2021-03-27 14:08:31,ARQUITECTURA E INGENIERIA ACONCAGUA SPA,Video Call,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.,27-03-2021 14:08
4,82413,8456211-4,8456211-4,Titular,Doctor Online,2021-04-09 13:34:07,EMPRESA DE MONTAJES INDUSTRIALES SALFA S A,Llamada telefónica,Vida Security,SALFA GESTION INGENIERIA S.A.,09-04-2021 13:34
...,...,...,...,...,...,...,...,...,...,...,...
45,102021,22615307-1,13265231-7,Hijo/a,Orientación Hospitalaria,2021-06-03 17:42:26,SERVICIO DE VTA INMOB LTDA.,Llamada telefónica,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.,03-06-2021 17:42
46,102213,18162191-5,18162191-5,Titular,Doctor Online,2021-06-04 02:40:44,SALFA GESTION INGENIERIA S.A.,Video Call,Vida Security,SALFA GESTION INGENIERIA S.A.,04-06-2021 02:40
47,102588,22615307-1,13265231-7,Hijo/a,Programa de Psicología,2021-06-04 21:33:22,SERVICIO DE VTA INMOB LTDA.,Llamada telefónica,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.,04-06-2021 21:33
48,102678,16716097-2,16716097-2,Titular,Programa de Psicología,2021-06-05 14:01:30,ACONCAGUA GESTION INMOBILIARIA S.A.,Llamada telefónica,Vida Security,ACONCAGUA GESTION INMOBILIARIA S.A.,05-06-2021 14:01


In [91]:
conchaytoro.to_excel(archivo, index=False)